In [1]:
import pandas as pd
import sqlite3 as lite
from sqlite3 import Error
from pathlib import Path
import ast
import tqdm

In [2]:
def create_connection(db_file):
    """
    create a connection to sqlite3 database
    """
    conn = None
    try:
        conn = lite.connect(db_file, timeout=10)  # connection via sqlite3
        # engine = sa.create_engine('sqlite:///' + db_file)  # connection via sqlalchemy
        # conn = engine.connect()
    except Error as e:
        print(e)
    return conn


DATA_PATH = Path.cwd().parents[0] / 'Data'

conn = create_connection(DATA_PATH / "CVEfixes.db")

In [3]:
df_commit = pd.read_sql('SELECT * FROM commits', con=conn)
df_file = pd.read_sql('SELECT * FROM file_change', con=conn)
df_method = pd.read_sql('SELECT * FROM method_change', con=conn)
df_cve = pd.read_sql('SELECT * FROM cve', con=conn)
df_fixes = pd.read_sql('SELECT * FROM fixes', con=conn)
df_cwe_class = pd.read_sql('SELECT * FROM cwe_classification', con=conn)
df_cwe = pd.read_sql('SELECT * FROM cwe', con=conn)
df_repo = pd.read_sql('SELECT * FROM repository', con = conn)

Count changed files per commit:

In [4]:
file_change_per_commit = df_file.groupby(['hash']).size().reset_index(name='file_changes')
print(file_change_per_commit.head())

file_change_per_commit.groupby('file_changes').size()

                                       hash  file_changes
0  000030feb7a30f193197f1aab8a7b04a26b42735             1
1  0002d106a6cd35cb0a6fe03246531a4e3f32c9d0             1
2  0010d28de1b15d51db3976080e26357fa7144436             1
3  00194f5fe462123f70b0bae7987317b52898b868             1
4  001a3278b5572e52c0ecac0bd1157bf2599502b7             1


file_changes
1       3699
2       1409
3        813
4        406
5        248
        ... 
402        1
611        1
890        1
1776       1
2142       1
Length: 79, dtype: int64

Find local fixes:

In [5]:
# only consider commits with single file change

single_change = file_change_per_commit[file_change_per_commit['file_changes']==1]
filtered = df_file[df_file['hash'].isin(single_change['hash'])]
filtered.head()
#filtered.groupby('programming_language').size().sort_values(ascending=False).head(10)


,file_change_id,hash,filename,old_path,new_path,change_type,diff,diff_parsed,num_lines_added,num_lines_deleted,code_after,code_before,nloc,complexity,token_count,programming_language
19,174571154001437,04906bd5de2f220bf100b605dad37b4a1d9a91a6,saver.cpp,kscreensaver/saver.cpp,kscreensaver/saver.cpp,ModificationType.MODIFY,"@@ -151,6 +151,7 @@ void KPasswordDlg::keyPres...","{'added': [(154, ' timer.stop();')]...",1,0,#include <pwd.h>\n#include <unistd.h>\n#includ...,#include <pwd.h>\n#include <unistd.h>\n#includ...,197,34,1132,C++
20,47598748964815,c5be6209311d4a8f10fda37d0d3f876c1b33b77b,svr_principal.c,src/lib/kadm5/srv/svr_principal.c,src/lib/kadm5/srv/svr_principal.c,ModificationType.MODIFY,"@@ -186,7 +186,7 @@ check_1_6_dummy(kadm5_prin...","{'added': [(189, ' if (password == NULL || ...",1,1,/* -*- mode: c; c-basic-offset: 4; indent-tabs...,/* -*- mode: c; c-basic-offset: 4; indent-tabs...,1655.0,439.0,10633.0,C
21,164057968633021,cf1a0c411b2668c57c41e9c4efd15ba17b6b322c,schpw.c,src/kadmin/server/schpw.c,src/kadmin/server/schpw.c,ModificationType.MODIFY,"@@ -52,7 +52,7 @@ process_chpw_request(krb5_co...","{'added': [(55, ' goto bailout;'), (70,...",4,4,/* -*- mode: c; c-basic-offset: 4; indent-tabs...,/* -*- mode: c; c-basic-offset: 4; indent-tabs...,363.0,54.0,2230.0,C
22,179334716148121,f18ddf5d82de0ab7591a36e465bc24225776940f,spnego_mech.c,src/lib/gssapi/spnego/spnego_mech.c,src/lib/gssapi/spnego/spnego_mech.c,ModificationType.MODIFY,"@@ -818,7 +818,6 @@ init_ctx_reselect(OM_uint3...","{'added': [], 'deleted': [(821, '\tgeneric_gss...",0,1,"/*\n * Copyright (C) 2006,2008 by the Massachu...","/*\n * Copyright (C) 2006,2008 by the Massachu...",3104.0,512.0,15617.0,C
23,179231907027180,524688ce87a15fc75f87efc8c039ba4c7d5c197b,spnego_mech.c,src/lib/gssapi/spnego/spnego_mech.c,src/lib/gssapi/spnego/spnego_mech.c,ModificationType.MODIFY,"@@ -1468,7 +1468,7 @@ acc_ctx_cont(OM_uint32 *...","{'added': [(1471, '\tif (REMAIN == 0 || REMAIN...",1,1,"/*\n * Copyright (C) 2006,2008 by the Massachu...","/*\n * Copyright (C) 2006,2008 by the Massachu...",3104.0,513.0,15621.0,C


In [6]:
df_local_diff = filtered.copy() #df_file[df_file.programming_language=='C'].copy()
df_local_diff['diff_added'] = df_local_diff.apply(lambda row: ast.literal_eval(row.diff_parsed)['added'], axis=1)
df_local_diff['diff_deleted'] = df_local_diff.apply(lambda row: ast.literal_eval(row.diff_parsed)['deleted'], axis=1)
df_local_diff = df_local_diff.reset_index(drop=True)

In [7]:
def is_local_fix(row):
    added = row['diff_added']
    deleted = row['diff_deleted']

    if added:
        first_add = added[0][0]
        current = first_add
        for line in added[1:]:
            current += 1
            if line[0] > current:
                return False 
         
        if deleted:
            first_del = deleted[0][0]
            # add & delete at same position?
            if first_del >= first_add and first_del <= first_add + len(added): # check me
                current = first_del
                for line in deleted[1:]:
                    current += 1
                    if line[0] > current:
                         return False 
                return True
            else:
                return False 
        return True
         
    # nothing added, deletion only
    first_del = deleted[0][0]
    current = first_del
    for line in deleted[1:]:
        current += 1
        if line[0] > current:
            return False    
    return True

df_local_diff = df_local_diff[df_local_diff.apply(is_local_fix, axis=1)]
df_local_diff = df_local_diff.reset_index(drop=True)
# 1654 commits w local single file change

df_local_diff.groupby('programming_language').size().sort_values(ascending=False)

programming_language
C                   731
PHP                 310
C++                 204
JavaScript           85
Objective-C          56
Markdown             38
Ruby                 38
Python               36
Java                 32
TypeScript           21
HTML                 18
Perl                 18
Go                   14
Shell                12
C#                    7
Scala                 7
Rust                  6
SQL                   5
CoffeeScript          4
unknown               3
Swift                 3
TeX                   1
Batchfile             1
Lua                   1
Jupyter Notebook      1
Haskell               1
Matlab                1
dtype: int64

In [9]:
df_diff = df_file.copy() # all files with local changes
df_diff['diff_added'] = df_diff.apply(lambda row: ast.literal_eval(row.diff_parsed)['added'], axis=1)
df_diff['diff_deleted'] = df_diff.apply(lambda row: ast.literal_eval(row.diff_parsed)['deleted'], axis=1)


df_diff = df_diff[df_diff.apply(is_local_fix, axis=1)]
df_diff = df_diff.reset_index(drop=True)
# 17329 files w local change
df_diff.groupby('programming_language').size().sort_values(ascending=False)

programming_language
unknown             3792
PHP                 3468
C                   2208
Markdown            1239
JavaScript          1071
C++                  626
Ruby                 573
HTML                 558
Java                 498
Shell                461
Python               460
TypeScript           338
SQL                  297
Go                   250
Batchfile            246
CoffeeScript         162
CSS                  152
Lua                  141
C#                   130
Perl                 114
Objective-C          107
Scala                 96
TeX                   73
Haskell               69
Rust                  45
Jupyter Notebook      36
Matlab                34
PowerShell            33
Swift                 30
R                     12
Erlang                 6
None                   4
dtype: int64

In [10]:
def inspect_change(row):
    for col in ['diff_added', 'diff_deleted']:
        print(col, ":", row[col])

Cut files into prompt, vul, fix, remainder:

In [12]:

def create_sample(row):
    lines_before = row['code_before'].splitlines(True)
    lines_after = row['code_after'].splitlines(True)
    
    if row['diff_added']:
        first_add = row['diff_added'][0][0]
        last_add = row['diff_added'][-1][0]
        first_change = first_add
        last_change = last_add
    if row['diff_deleted']:
        first_del = row['diff_deleted'][0][0]
        last_del = row['diff_deleted'][-1][0]
        if row['diff_added']:
            first_change = min(first_change, first_del)
            last_change = max(last_add, last_del)
            if first_add == first_del and last_add == last_del:
                vul = "".join(lines_before[first_change-1:last_change])
                patch = "".join(lines_after[first_change-1:last_change])
                remainder = "".join(lines_after[last_change:])
            else:
                #inspect_change(row)
                vul = "".join(lines_before[first_del-1:last_del])
                patch = "".join(lines_after[first_add-1:last_add])
                remainder = "".join(lines_before[last_del:])
        else:
            first_change = first_del
            last_change = last_del
            vul = "".join(lines_before[first_del-1:last_del])
            patch = ""
            remainder = "".join(lines_before[last_del:])
    else:
        vul = ""
        patch = "".join(lines_after[first_add-1:last_add])
        remainder = "".join(lines_after[last_add:])

    
    row.loc['prompt'] = "".join(lines_before[:first_change-1]) # the line before the first change is the prompt.
    row.loc['target_vul'] = vul
    row.loc['target_patch'] = patch
    row.loc['remainder'] = remainder

    assert row.loc['prompt'] + row.loc['target_patch'] + row.loc['remainder'] == row.loc['code_after']
    if not row.loc['prompt'] + row.loc['target_vul'] + row.loc['remainder'] == row.loc['code_before']:
        print(row.loc['code_before'])
        print('***')
        print(row.loc['prompt'] + '\n***\n' + row.loc['target_vul'] + '\n***\n' + row.loc['remainder'])

#df_local_diff['programming_language']== 'python'

df_local_diff[df_local_diff['programming_language']== 'Python'].apply(create_sample, axis=1)


0       False
1       False
2       False
3       False
4       False
        ...  
1649    False
1650    False
1651    False
1652    False
1653    False
Name: programming_language, Length: 1654, dtype: bool

In [ ]:
#df_local_diff.head()

for row in df_local_diff.iterrows():
    print(row['code_before'])
    print('*'*10)
    print(row['diff_parsed'])

TypeError: tuple indices must be integers or slices, not str

: 

In [ ]:
print('Fixes:', df_fixes.columns)
print('File changes:', df_file.columns)

for row in df_file['diff_parsed'][:5]:
    print('added:',
            len(ast.literal_eval(row)['added']),
            'deleted:', 
            len(ast.literal_eval(row)['deleted']))



Fixes: Index(['cve_id', 'hash', 'repo_url'], dtype='object')
File changes: Index(['file_change_id', 'hash', 'filename', 'old_path', 'new_path',
       'change_type', 'diff', 'diff_parsed', 'num_lines_added',
       'num_lines_deleted', 'code_after', 'code_before', 'nloc', 'complexity',
       'token_count', 'programming_language'],
      dtype='object')
added: 52 deleted: 0
added: 28 deleted: 10
added: 5 deleted: 3
added: 8 deleted: 4
added: 1 deleted: 1


: 

In [ ]:

df_c_diff = df_file[df_file.programming_language=='C'].copy()
df_c_diff['diff_added'] = df_c_diff.apply(lambda row: ast.literal_eval(row.diff_parsed)['added'], axis=1)
df_c_diff['diff_deleted'] = df_c_diff.apply(lambda row: ast.literal_eval(row.diff_parsed)['deleted'], axis=1)
df_c_diff = df_c_diff.reset_index(drop=True)
df_c_diff.head(5)

,file_change_id,hash,filename,old_path,new_path,change_type,diff,diff_parsed,num_lines_added,num_lines_deleted,code_after,code_before,nloc,complexity,token_count,programming_language,diff_added,diff_deleted
0,7461293048740,2864e767053317538feafa815046fff89e5a16be,dl-load.c,elf/dl-load.c,elf/dl-load.c,ModificationType.MODIFY,"@@ -149,21 +149,31 @@ local_strdup (const char...","{'added': [(152, ' const char *const start = ...",28,10,/* Map in a shared object's segments from the ...,/* Map in a shared object's segments from the ...,952.0,260.0,6592.0,C,"[(152, const char *const start = name;), (15...","[(158, /* $ORIGIN is not expanded for SU..."
1,253444729592566,2864e767053317538feafa815046fff89e5a16be,syslog.c,misc/syslog.c,misc/syslog.c,ModificationType.MODIFY,"@@ -177,10 +177,14 @@ vsyslog(pri, fmt, ap)\n ...","{'added': [(180, '\t\t/* Append a newline if n...",8,4,"/*\n * Copyright (c) 1983, 1988, 1993\n *\tThe...","/*\n * Copyright (c) 1983, 1988, 1993\n *\tThe...",229.0,42.0,1240.0,C,"[(180, \t\t/* Append a newline if necessary. ...","[(180, \t\t++v;), (181, \t\tv->iov_base = (cha..."
2,88393274694273,2864e767053317538feafa815046fff89e5a16be,regex.c,posix/regex.c,posix/regex.c,ModificationType.MODIFY,"@@ -164,46 +164,6 @@ char *realloc ();\n # de...","{'added': [(239, '#ifndef emacs'), (240, '/* H...",37,40,/* Extended regular expression matching and se...,/* Extended regular expression matching and se...,2789.0,805.0,14578.0,C,"[(239, #ifndef emacs), (240, /* How many chara...","[(167, /* How many characters in the character..."
3,55018883367054,2864e767053317538feafa815046fff89e5a16be,tst-getlogin.c,None,posix/tst-getlogin.c,ModificationType.ADD,"@@ -0,0 +1,58 @@\n+/* Copyright (C) 1999 Free ...","{'added': [(1, '/* Copyright (C) 1999 Free Sof...",58,0,/* Copyright (C) 1999 Free Software Foundation...,None,35.0,4.0,129.0,C,"[(1, /* Copyright (C) 1999 Free Software Found...",[]
4,156577410779052,2864e767053317538feafa815046fff89e5a16be,getlogin_r.c,sysdeps/unix/getlogin_r.c,sysdeps/unix/getlogin_r.c,ModificationType.MODIFY,"@@ -40,20 +40,20 @@ getlogin_r (name, name_len...","{'added': [(43, ' /* Get name of tty connecte...",16,16,/* Reentrant function to return the current lo...,/* Reentrant function to return the current lo...,46.0,5.0,195.0,C,"[(43, /* Get name of tty connected to fd 0. ...","[(43, {), (44, int d = __open (""/dev/tty..."


: 

In [ ]:
LINES_IN_TARGET = 1
df_diff['prompt'] = None
df_diff['target_vul'] = None
df_diff['target_patch'] = None
df_diff['remainder'] = None


def inspect_change(idx):
    for col in ['diff_parsed', 'diff_added', 'diff_deleted']:
        print(col, ":", df_diff[col][idx])


def first_change(idx):
    if df_diff['diff_added'][idx]:
        if df_diff['diff_deleted'][idx]:
            return min(df_diff['diff_added'][idx][0][0], df_diff['diff_deleted'][idx][0][0])
        else:
            return df_diff['diff_added'][idx][0][0]
    else:
        return df_diff['diff_deleted'][idx][0][0]


# add prompt for ltr generation + vulnerable & fixed target
for i in range(len(df_diff['code_before'])):
    changed = first_change(i)
    lines_before = df_diff['code_before'][i].splitlines(True)
    lines_after = df_diff['code_after'][i].splitlines(True)

    df_diff.loc[i, 'prompt'] = "".join(lines_before[:changed-1]) # the line before the first change is the prompt.
    df_diff.loc[i, 'target_vul'] = "".join(lines_before[changed-1:min(changed+LINES_IN_TARGET-1, len(lines_before))]) 
    df_diff.loc[i, 'target_patch'] = "".join(lines_after[changed-1:min(changed+LINES_IN_TARGET-1, len(lines_after))])
    df_diff.loc[i, 'remainder'] = "".join(lines_after[changed: len(lines_after)])

df_diff.head(5)

,file_change_id,hash,filename,old_path,new_path,change_type,diff,diff_parsed,num_lines_added,num_lines_deleted,...,nloc,complexity,token_count,programming_language,diff_added,diff_deleted,prompt,target_vul,target_patch,remainder
0,7461293048740,2864e767053317538feafa815046fff89e5a16be,dl-load.c,elf/dl-load.c,elf/dl-load.c,ModificationType.MODIFY,"@@ -149,21 +149,31 @@ local_strdup (const char...","{'added': [(152, ' const char *const start = ...",28,10,...,952.0,260.0,6592.0,C,"[(152, const char *const start = name;), (15...","[(158, /* $ORIGIN is not expanded for SU...",/* Map in a shared object's segments from the ...,size_t cnt = 0;\n,const char *const start = name;\n,size_t cnt = 0;\n\n do\n {\n size_t...
1,253444729592566,2864e767053317538feafa815046fff89e5a16be,syslog.c,misc/syslog.c,misc/syslog.c,ModificationType.MODIFY,"@@ -177,10 +177,14 @@ vsyslog(pri, fmt, ap)\n ...","{'added': [(180, '\t\t/* Append a newline if n...",8,4,...,229.0,42.0,1240.0,C,"[(180, \t\t/* Append a newline if necessary. ...","[(180, \t\t++v;), (181, \t\tv->iov_base = (cha...","/*\n * Copyright (c) 1983, 1988, 1993\n *\tThe...",\t\t++v;\n,\t\t/* Append a newline if necessary. */\n,\t\tif (buf[bufsize - 1] != '\n')\n\t\t {\n\t...
2,88393274694273,2864e767053317538feafa815046fff89e5a16be,regex.c,posix/regex.c,posix/regex.c,ModificationType.MODIFY,"@@ -164,46 +164,6 @@ char *realloc ();\n # de...","{'added': [(239, '#ifndef emacs'), (240, '/* H...",37,40,...,2789.0,805.0,14578.0,C,"[(239, #ifndef emacs), (240, /* How many chara...","[(167, /* How many characters in the character...",/* Extended regular expression matching and se...,/* How many characters in the character set. ...,#endif /* not emacs */\n,"\n/* Get the interface, including the syntax ..."
3,55018883367054,2864e767053317538feafa815046fff89e5a16be,tst-getlogin.c,None,posix/tst-getlogin.c,ModificationType.ADD,"@@ -0,0 +1,58 @@\n+/* Copyright (C) 1999 Free ...","{'added': [(1, '/* Copyright (C) 1999 Free Sof...",58,0,...,35.0,4.0,129.0,C,"[(1, /* Copyright (C) 1999 Free Software Found...",[],,None,/* Copyright (C) 1999 Free Software Foundation...,This file is part of the GNU C Library.\n\n...
4,156577410779052,2864e767053317538feafa815046fff89e5a16be,getlogin_r.c,sysdeps/unix/getlogin_r.c,sysdeps/unix/getlogin_r.c,ModificationType.MODIFY,"@@ -40,20 +40,20 @@ getlogin_r (name, name_len...","{'added': [(43, ' /* Get name of tty connecte...",16,16,...,46.0,5.0,195.0,C,"[(43, /* Get name of tty connected to fd 0. ...","[(43, {), (44, int d = __open (""/dev/tty...",/* Reentrant function to return the current lo...,{\n,/* Get name of tty connected to fd 0. Retur...,if fd 0 isn't open. Note that a lot of d...


: 

In [ ]:
output_cols = ['prompt', 'target_vul', 'target_patch', 'remainder']
df_diff[output_cols].to_csv(DATA_PATH / 'ltr_samples.csv', index=False)

: 

In [ ]:
def inspect_entry(i:int):
    print(df_diff['prompt'][i] + 
    "VUL:"+"~"*20 +"\n" + 
    df_diff['target_vul'][i] + 
    "PATCH:"+"~"*20 +"\n" + 
    df_diff['target_patch'][i] +
    "REMAINDER:"+"~"*20 +"\n" + 
    df_diff['remainder'][i])

inspect_entry(0)

for i in range(len(df_diff['prompt'])):
    assert df_diff['prompt'][i] + df_diff['target_patch'][i] + df_diff['remainder'][i] == df_diff['code_after'][i]
    assert df_diff['prompt'][i] + df_diff['target_vul'][i] + df_diff['remainder'][i] == df_diff['code_before'][i], f"Error in line {i}"

/* Map in a shared object's segments from the file.
   Copyright (C) 1995, 1996, 1997, 1998, 1999 Free Software Foundation, Inc.
   This file is part of the GNU C Library.

   The GNU C Library is free software; you can redistribute it and/or
   modify it under the terms of the GNU Library General Public License as
   published by the Free Software Foundation; either version 2 of the
   License, or (at your option) any later version.

   The GNU C Library is distributed in the hope that it will be useful,
   but WITHOUT ANY WARRANTY; without even the implied warranty of
   MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU
   Library General Public License for more details.

   You should have received a copy of the GNU Library General Public
   License along with the GNU C Library; see the file COPYING.LIB.  If not,
   write to the Free Software Foundation, Inc., 59 Temple Place - Suite 330,
   Boston, MA 02111-1307, USA.  */

#include <elf.h>
#include <errno.h>
#includ

AssertionError: Error in line 0

: 

In [ ]:
df_method.head(5)
print(len(df_method))
method_diff = df_method[['file_change_id', 'signature']].value_counts().reset_index(name='count')
method_diff['code_before'] = None
method_diff['code_after'] = None
for idx in tqdm(df_method.index):
    if idx % 1000 == 0:
        print(idx)
    if df_method.loc[idx, 'before_change'] == 'True':
        method_diff.loc[(method_diff['file_change_id']==df_method.loc[idx, 'file_change_id']) & 
                    (method_diff['signature']==df_method.loc[idx, 'signature']), 'code_before'] = df_method.loc[idx,'code']

    else:
        method_diff.loc[(method_diff['file_change_id']==df_method.loc[idx, 'file_change_id']) & 
                    (method_diff['signature']==df_method.loc[idx, 'signature']), 'code_after'] = df_method.loc[idx,'code']
        
    
method_diff.head(5)

: 

In [ ]:
method_diff.to_csv(DATA_PATH / 'methods.csv', index=True)

: 

In [ ]:
from typing import List, Dict, Tuple

# See https://github.com/ishepard/pydriller/blob/master/pydriller/domain/commit.py

def get_line_numbers(line: str) -> Tuple[int, int]:
    token = line.split(" ")
    numbers_old_file = token[1]
    numbers_new_file = token[2]
    delete_line_number = (
            int(numbers_old_file.split(",")[0].replace("-", "")) - 1
    )
    additions_line_number = int(numbers_new_file.split(",")[0]) - 1
    return delete_line_number, additions_line_number

def to_diff_parsed(diff:str) -> Dict[str, List[Tuple[int, str]]]:
    """
    Returns a dictionary with the added and deleted lines.
    The dictionary has 2 keys: "added" and "deleted", each containing the
    corresponding added or deleted lines. For both keys, the value is a
    list of Tuple (int, str), corresponding to (number of line in the file,
    actual line).
    :return: Dictionary
    """
    lines = diff.split("\n")
    modified_lines = {
        "added": [],
        "deleted": [],
    }  # type: Dict[str, List[Tuple[int, str]]]

    count_deletions = 0
    count_additions = 0

    for line in lines:
        line = line.rstrip()
        count_deletions += 1
        count_additions += 1

        if line.startswith("@@"):
            count_deletions, count_additions = get_line_numbers(line)

        if line.startswith("-"):
            modified_lines["deleted"].append((count_deletions, line[1:]))
            count_additions -= 1

        if line.startswith("+"):
            modified_lines["added"].append((count_additions, line[1:]))
            count_deletions -= 1

        if line == r"\ No newline at end of file":
            count_deletions -= 1
            count_additions -= 1

    return modified_lines

: 

In [ ]:
query = """
SELECT cv.cve_id, f.filename, f.num_lines_added, f.num_lines_deleted, f.code_before, f.code_after, cc.cwe_id 
FROM file_change f, commits c, fixes fx, cve cv, cwe_classification cc
WHERE f.hash = c.hash 
AND c.hash = fx.hash 
AND fx.cve_id = cv.cve_id 
AND cv.cve_id = cc.cve_id 
AND f.num_lines_added<=1 
AND f.num_lines_deleted<=1;
"""
single_line_fixes = pd.read_sql_query(query, conn)
print(len(single_line_fixes))
single_line_fixes.head(5)

6876


,cve_id,filename,num_lines_added,num_lines_deleted,code_before,code_after,cwe_id
0,CVE-1999-0199,Makefile,1,1,"# Copyright (C) 1991,92,93,94,95,96,97,98,99 F...","# Copyright (C) 1991,92,93,94,95,96,97,98,99 F...",CWE-252
1,CVE-1999-0731,saver.cpp,1,0,#include <pwd.h>\n#include <unistd.h>\n#includ...,#include <pwd.h>\n#include <unistd.h>\n#includ...,NVD-CWE-Other
2,CVE-2012-1013,svr_principal.c,1,1,/* -*- mode: c; c-basic-offset: 4; indent-tabs...,/* -*- mode: c; c-basic-offset: 4; indent-tabs...,NVD-CWE-Other
3,CVE-2014-4343,spnego_mech.c,0,1,"/*\n * Copyright (C) 2006,2008 by the Massachu...","/*\n * Copyright (C) 2006,2008 by the Massachu...",CWE-415
4,CVE-2014-4344,spnego_mech.c,1,1,"/*\n * Copyright (C) 2006,2008 by the Massachu...","/*\n * Copyright (C) 2006,2008 by the Massachu...",CWE-476


: 